In [2]:
import dotenv
import os
dotenv.load_dotenv()

True

In [3]:

OPENAI_API_KEY=os.environ.get("OPENAI_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder ,PromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
import uuid
from langchain_core.chat_history import InMemoryChatMessageHistory,BaseChatMessageHistory

In [143]:
model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0.2)

In [144]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a Medical Bot. Your primary role is to provide accurate and reliable medical information, offer guidance on symptoms, treatments, and general health concerns, and assist with medical inquiries. Always ensure your advice is based on established medical knowledge, and encourage users to consult healthcare professionals for personalized medical care.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)
parser=StrOutputParser()

chain = prompt | model | parser

In [156]:
def get_chat_history(session_id: str) :
    chat_histories= {}
    if session_id not in chat_histories:
        chat_histories[session_id] = InMemoryChatMessageHistory()
    return chat_histories[session_id]

In [157]:
def get_chat_history(self,session_id: str):
    self.chat_histories={}  
    if session_id not in self.chat_histories:
        self.chat_histories[session_id] = InMemoryChatMessageHistory()
        print(self.chat_histories)
    return self.chat_histories[session_id]   


In [12]:
chat_histories = {}
class Query():
       
    def get_chat_history(self,session_id: str):  
        if session_id not in chat_histories:
            chat_histories[session_id] = InMemoryChatMessageHistory()
            print(chat_histories)
        return chat_histories[session_id]  
    def handle_user_input(self,user_input: str, session_id: str = None):
        model = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0.2)

        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "You are a Medical Bot. Your primary role is to provide accurate and reliable medical information, offer guidance on symptoms, treatments, and general health concerns, and assist with medical inquiries. Always ensure your advice is based on established medical knowledge, and encourage users to consult healthcare professionals for personalized medical care.",
                ),
                MessagesPlaceholder(variable_name="chat_history"),
                ("human", "{input}"),
            ]
        )

        parser=StrOutputParser()
        chain = prompt | model | parser

        if session_id is None: 
            prompt_sid=PromptTemplate(
            input_variables=["user_input"],
            template="Write a concise summary of the following:{user_input}.The summary should be of 2 to 3 words"
                )

            parse= prompt_sid.format(user_input=user_input)
            s_id = model.invoke(parse)
            session_id = s_id.content
            #print(session_id)
        config = {'configurable': {'session_id': session_id}}
        print(config)

        chain_with_message_history = RunnableWithMessageHistory(
        chain,
        lambda session_id: self.get_chat_history(session_id),
        input_messages_key="input",
        history_messages_key="chat_history",
        )

        payload = {
            "input": user_input,
            "chat_history": self.get_chat_history(session_id)
        }

        response = chain_with_message_history.invoke(payload,config=config)
        return session_id, response


In [13]:
user_input="I need prescription for fever "
q=Query()
session_id,response=q.handle_user_input(user_input)
response

user_input="I need to consult a doctot "
q=Query()
session_id,response=q.handle_user_input(user_input)
response

{'configurable': {'session_id': 'Prescription for fever'}}
{'Prescription for fever': InMemoryChatMessageHistory(messages=[])}
{'configurable': {'session_id': 'Consult doctor.'}}
{'Prescription for fever': InMemoryChatMessageHistory(messages=[HumanMessage(content='I need prescription for fever '), AIMessage(content="I'm sorry, but as a Medical Bot, I am not able to provide prescriptions. If you are experiencing a fever, it's important to consult with a healthcare provider in person or through a telehealth appointment. A healthcare professional can assess your symptoms, determine the underlying cause of the fever, and provide appropriate treatment or medication if necessary. In the meantime, you can try to manage your fever by staying hydrated, getting plenty of rest, and taking over-the-counter fever-reducing medications like acetaminophen or ibuprofen following the recommended dosage instructions. If your fever is persistent, accompanied by severe symptoms, or if you have any underlyi

"If you need to consult a doctor, it's important to schedule an appointment with a healthcare provider. Depending on your location and healthcare system, you can contact your primary care physician, a specialist, or visit an urgent care center or hospital if it's an emergency. It's essential to seek professional medical advice for any health concerns you may have. If you need help finding a doctor or have questions about your health, feel free to ask for guidance."

In [6]:
user_input="I need to consult a doctot "
q=Query()
session_id,response=q.handle_user_input(user_input)
response

{'configurable': {'session_id': 'Consult doctor'}}
{'Consult doctor': InMemoryChatMessageHistory(messages=[])}


"If you need to consult a doctor, it's important to schedule an appointment with a healthcare provider. They can assess your symptoms, provide a diagnosis, and recommend appropriate treatment. If it's an emergency, please seek immediate medical attention by calling emergency services or going to the nearest hospital. If you have any specific medical questions or concerns, feel free to ask, and I'll do my best to provide information to help you."

AttributeError: type object 'Query' has no attribute 'chat_histories'